# Importing machine learning packages and define the grid search steps

In [1]:
###########import packages##########
import pickle
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf 
from sklearn.model_selection import GridSearchCV
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.multioutput import MultiOutputRegressor
import catboost
import lightgbm
###########fix random seed for reproducability##########
seed=1
np.random.seed(seed)
###########wrapping root mean square error for later calls##########
def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())
###########loading data##########
fdata=pd.read_csv('databaseML.csv',encoding="gbk")
raw_data=fdata.loc[:,[                     
                      'Pt in metal element',#0
                      'Co in metal element',#1
                      'total metal mass ratio',#2
                      'C',#3
                      'Particle diameter',#4
                      'support BET surface area' ,#5
                      'Reduction Temperature',#6
                      'Reduction Time',#7
                      'Annealing Temperature',#8
                      'ECSA',#9
                      'Mass Activity',#10
                      'IC',#11
                      'Area cm2',#12
                      'Cathodic Loading Amount',#13
                      'Anodic Platinum Loading Amount',#14
                      'Anodic catalyst type',#15
                      'membrane thickness',#16
                      'Hot Press Temperature',#17
                      'Hot Press Time min',#18
                      'Hot Press Pressure Mpa',#19
                      'Humidity',#20
                      'GDE for 1',#21
                      'celltemp',#22
                      'Flowing rate of H2',#23
                      'flowing rate of cathode gas',#24
                      'Back Pressure Mpa',#25
                      'Cathode gas oxygen ratio',#26
                      'Open Circuit Voltage V',#27
                      'Voltage 100mA cm0',
                      'Voltage 200mA cm0',
                      'Voltage 300mA cm0',
                      'Voltage 400mA cm0',
                      'Voltage 500mA cm0',
                      'Voltage 600mA cm0',
                      'Voltage 700mA cm0',
                      'Voltage 800mA cm0',
                      'Voltage 900mA cm0',
                      'Voltage 1000mA cm0',
                      'Voltage 1100mA cm0',
                      'Voltage 1200mA cm0',
                      'Voltage 1300mA cm0',
                      'Voltage 1400mA cm0',
                      'Voltage 1500mA cm0',
                      'Voltage 1600mA cm0',
                      'Voltage 1700mA cm0',
                      'Voltage 1800mA cm0',
                      'Voltage 1900mA cm0',
                      'Voltage 2000mA cm0',
                      'Voltage 2100mA cm0',
                      'Voltage 2200mA cm0',
                      'Voltage 2300mA cm0',
                      'Voltage 2400mA cm0',
                      'Voltage 2500mA cm0',
                      'Voltage 2600mA cm0',
                      'Voltage 2700mA cm0',
                      'Voltage 2800mA cm0',
                      'Voltage 2900mA cm0',
                      'Voltage 3000mA cm0'#57
                        ]]
###########handling missing values##########
median_raw_data=raw_data.median()
dict_median_raw_data=median_raw_data.to_dict()
data=raw_data.fillna(dict_median_raw_data)
###########data standardization##########
standardized_data = (data-np.mean(data,axis=0))/np.std(data,axis=0)
###########train test splitting##########
raw_param_global=data.iloc[:,0:27]
raw_power_global=data.iloc[:,27:]
#input allocation
median=standardized_data.median()
raw_median=data.median()
user_input1=raw_median[0:27]
standardized_user_input=(user_input1-np.mean(data,axis=0)[0:27])/np.std(data,axis=0)[0:27]
###########defining a wrapper function for later call from each machine learning algorithms##########
   
def try_different_method(model,param,algorithm_name):
    voltage_result_list_each_235=[]
    voltage_result_list_each_381=[]
    voltage_result_list_each_456=[]
    voltage_result_list_each_619=[]
    voltage_result_list_each_582=[]
    voltage_result_list_each_583=[]
    for i in range(27,58,1):
        raw_param=standardized_data.iloc[:,0:27]
        raw_power=standardized_data.iloc[:,i]
        X_train=raw_param.drop(index=[235,381,456,619,582,583])
        y_train=raw_power.drop(index=[235,381,456,619,582,583])
        X_test_235=standardized_data.iloc[235,0:27]
        X_test_381=standardized_data.iloc[381,0:27]
        X_test_456=standardized_data.iloc[456,0:27]
        X_test_619=standardized_data.iloc[619,0:27]
        X_test_582=standardized_data.iloc[582,0:27]
        X_test_583=standardized_data.iloc[583,0:27]
        np.random.seed(1)
        grid = GridSearchCV(model,param_grid=param,cv=5)
        grid.fit(X_train,y_train)
        print('Best Regressor:',grid.best_params_,'Best Score:', grid.best_score_) 
        best_model=grid.best_estimator_
        best_model.fit(X_train,y_train)
        ###########get the prediction##########
        data_test_input_235=X_test_235
        data_test_input_235=pd.DataFrame(data_test_input_235)
        data_test_input_235=data_test_input_235.T
        data_test_param_235=data_test_input_235.values.astype(np.float32)
        print(data_test_param_235.shape)
        predict_ann_235_each= best_model.predict(data_test_param_235)
        voltage_result_list_each_235.append(predict_ann_235_each[0])
        
        data_test_input_381=X_test_381
        data_test_input_381=pd.DataFrame(data_test_input_381)
        data_test_input_381=data_test_input_381.T
        data_test_param_381=data_test_input_381.values.astype(np.float32)
        predict_ann_381_each= best_model.predict(data_test_param_381)
        voltage_result_list_each_381.append(predict_ann_381_each[0])
        
        data_test_input_456=X_test_456
        data_test_input_456=pd.DataFrame(data_test_input_456)
        data_test_input_456=data_test_input_456.T
        data_test_param_456=data_test_input_456.values.astype(np.float32)
        predict_ann_456_each= best_model.predict(data_test_param_456)
        voltage_result_list_each_456.append(predict_ann_456_each[0])
        
        data_test_input_619=X_test_619
        data_test_input_619=pd.DataFrame(data_test_input_619)
        data_test_input_619=data_test_input_619.T
        data_test_param_619=data_test_input_619.values.astype(np.float32)
        predict_ann_619_each= best_model.predict(data_test_param_619)
        voltage_result_list_each_619.append(predict_ann_619_each[0])
        
        data_test_input_582=X_test_582
        data_test_input_582=pd.DataFrame(data_test_input_582)
        data_test_input_582=data_test_input_582.T
        data_test_param_582=data_test_input_582.values.astype(np.float32)
        predict_ann_582_each= best_model.predict(data_test_param_582)
        voltage_result_list_each_582.append(predict_ann_582_each[0])

        data_test_input_583=X_test_583
        data_test_input_583=pd.DataFrame(data_test_input_583)
        data_test_input_583=data_test_input_583.T
        data_test_param_583=data_test_input_583.values.astype(np.float32)
        predict_ann_583_each= best_model.predict(data_test_param_583)
        voltage_result_list_each_583.append(predict_ann_583_each[0])
        
    ###########transfer the prediction to voltage values##########235
    voltage_result_list_235=np.std(raw_power_global,axis=0).T.values*voltage_result_list_each_235+np.mean(raw_power_global,axis=0).T.values
    print(voltage_result_list_235)
    ###########computing output power##########
    x_current=np.arange(0,3100,100)
    predict_power_235=x_current*voltage_result_list_235
    predict_power_235=predict_power_235.T
    ###########real values in literature##########
    curve_real_235=data.iloc[235,27:]
    predict_power_real_235=x_current*data.iloc[235,27:].values
    ###########computing R2##########
    real_current_series_235=pd.Series(data.iloc[235,27:48].values)
    voltage_result_series_235=pd.Series(voltage_result_list_235[0:21])
    corr_ann_235 = round(voltage_result_series_235.corr(real_current_series_235), 4)
    rmse_val_235= rmse(data.iloc[235,27:48].values,voltage_result_list_235[0:21])
    print('ANN_235',corr_ann_235,'RMSE_235',rmse_val_235)   
    
    ###########transfer the prediction to voltage values##########381
    voltage_result_list_381=np.std(raw_power_global,axis=0).T.values*voltage_result_list_each_381+np.mean(raw_power_global,axis=0).T.values
    print(voltage_result_list_381)
    ###########computing output power##########
    x_current=np.arange(0,3100,100)
    predict_power_381=x_current*voltage_result_list_381
    predict_power_381=predict_power_381.T
    ###########real values in literature##########
    curve_real_381=data.iloc[381,27:]
    predict_power_real_381=x_current*data.iloc[381,27:].values
    ###########computing R2##########
    real_current_series_381=pd.Series(data.iloc[381,27:48].values)
    voltage_result_series_381=pd.Series(voltage_result_list_381[0:21])
    corr_ann_381 = round(voltage_result_series_381.corr(real_current_series_381), 4)
    rmse_val_381= rmse(data.iloc[381,27:48].values,voltage_result_list_381[0:21])
    print('ANN_381',corr_ann_381,'RMSE_381',rmse_val_381)    

    ###########transfer the prediction to voltage values##########456
    voltage_result_list_456=np.std(raw_power_global,axis=0).T.values*voltage_result_list_each_456+np.mean(raw_power_global,axis=0).T.values
    print(voltage_result_list_456)
    ###########computing output power##########
    x_current=np.arange(0,3100,100)
    predict_power_456=x_current*voltage_result_list_456
    predict_power_456=predict_power_456.T
    ###########real values in literature##########
    curve_real_456=data.iloc[456,27:]
    predict_power_real_456=x_current*data.iloc[456,27:].values
    ###########computing R2##########
    real_current_series_456=pd.Series(data.iloc[456,27:44].values)
    voltage_result_series_456=pd.Series(voltage_result_list_456[0:17])
    corr_ann_456 = round(voltage_result_series_456.corr(real_current_series_456), 4)
    rmse_val_456= rmse(data.iloc[456,27:44].values,voltage_result_list_456[0:17])  
    print('ANN_456',corr_ann_456,'RMSE_456',rmse_val_456)    

    ###########transfer the prediction to voltage values##########619
    voltage_result_list_619=np.std(raw_power_global,axis=0).T.values*voltage_result_list_each_619+np.mean(raw_power_global,axis=0).T.values
    print(voltage_result_list_619)
    ###########computing output power##########
    x_current=np.arange(0,3100,100)
    predict_power_619=x_current*voltage_result_list_619
    predict_power_619=predict_power_619.T
    ###########real values in literature##########
    curve_real_619=data.iloc[619,27:]
    predict_power_real_619=x_current*data.iloc[619,27:].values
    ###########computing R2##########
    real_current_series_619=pd.Series(data.iloc[619,27:48].values)
    voltage_result_series_619=pd.Series(voltage_result_list_619[0:21])
    corr_ann_619 = round(voltage_result_series_619.corr(real_current_series_619), 4)
    rmse_val_619= rmse(data.iloc[619,27:48].values,voltage_result_list_619[0:21])
    print('ANN_619',corr_ann_619,'RMSE_619',rmse_val_619)  
    
    voltage_result_list_582=np.std(raw_power_global,axis=0).T.values*voltage_result_list_each_582+np.mean(raw_power_global,axis=0).T.values
    print(voltage_result_list_582)
    ###########computing output power##########
    x_current=np.arange(0,3100,100)
    predict_power_582=x_current*voltage_result_list_582
    predict_power_582=predict_power_582.T
    ###########real values in literature##########
    curve_real_582=data.iloc[582,27:]
    predict_power_real_582=x_current*data.iloc[582,27:].values
    ###########computing R2##########
    real_current_series_582=pd.Series(data.iloc[582,27:53].values)
    voltage_result_series_582=pd.Series(voltage_result_list_582[0:26])
    corr_ann_582 = round(voltage_result_series_582.corr(real_current_series_582), 4)
    rmse_val_582= rmse(data.iloc[582,27:53].values,voltage_result_list_582[0:26])
    print('ANN_582',corr_ann_582,'RMSE_582',rmse_val_582)
    
    voltage_result_list_583=np.std(raw_power_global,axis=0).T.values*voltage_result_list_each_583+np.mean(raw_power_global,axis=0).T.values
    print(voltage_result_list_583)
    ###########computing output power##########
    x_current=np.arange(0,3100,100)
    predict_power_583=x_current*voltage_result_list_583
    predict_power_583=predict_power_583.T
    ###########real values in literature##########
    curve_real_583=data.iloc[583,27:]
    predict_power_real_583=x_current*data.iloc[583,27:].values
    ###########computing R2##########
    real_current_series_583=pd.Series(data.iloc[583,27:58].values)
    voltage_result_series_583=pd.Series(voltage_result_list_583[0:31])
    corr_ann_583 = round(voltage_result_series_583.corr(real_current_series_583), 4)
    rmse_val_583= rmse(data.iloc[583,27:58].values,voltage_result_list_583[0:31])
    print('ANN_583',corr_ann_583,'RMSE_583',rmse_val_583)


    ###########visualization##########
    fig1 = plt.figure()
    ax = fig1.add_subplot(111)
    ax.scatter(x_current[0:21],voltage_result_list_381[0:21],color='red',label='Predicted Voltage ')
    ax.plot(x_current[0:21],data.iloc[381,27:48].values,color='orange')
    ax.set_xlabel(u"Current(mA cm^-2)")
    ax.set_ylabel(u"Voltage(V)")
    ax2 = ax.twinx()
    ax2.set_ylabel(u'Power Density (mW cm^-2)')
    ax2.scatter(x_current[0:21],predict_power_381[0:21],color='green',label='Predicted Power Density ')
    ax2.plot(x_current[0:21],predict_power_real_381[0:21],color='blue')
    plt.legend()
    plt.savefig('%s CURVE 1.png' %(algorithm_name))
    print(voltage_result_list_381[0:21])
    print(data.iloc[381,27:48].values)
    print(predict_power_381[0:21])
    print(predict_power_real_381[0:21])
    
    fig2 = plt.figure()
    ax = fig2.add_subplot(111)
    ax.scatter(x_current[0:21],voltage_result_list_619[0:21],color='red',label='Predicted Voltage ')
    ax.plot(x_current[0:21],data.iloc[619,27:48].values,color='orange')
    ax.set_xlabel(u"Current(mA cm^-2)")
    ax.set_ylabel(u"Voltage(V)")
    ax2 = ax.twinx()
    ax2.set_ylabel(u'Power Density (mW cm^-2)')
    ax2.scatter(x_current[0:21],predict_power_619[0:21],color='green',label='Predicted Power Density ')
    ax2.plot(x_current[0:21],predict_power_real_619[0:21],color='blue')
    plt.legend()
    plt.savefig('%s CURVE 2.png' %(algorithm_name))
    print(voltage_result_list_619[0:21])
    print(data.iloc[619,27:48].values)
    print(predict_power_619[0:21])
    print(predict_power_real_619[0:21])

    fig3 = plt.figure()
    ax = fig3.add_subplot(111)
    ax.scatter(x_current[0:21],voltage_result_list_235[0:21],color='red',label='Predicted Voltage ')
    ax.plot(x_current[0:21],data.iloc[235,27:48].values,color='orange')
    ax.set_xlabel(u"Current(mA cm^-2)")
    ax.set_ylabel(u"Voltage(V)")
    ax2 = ax.twinx()
    ax2.set_ylabel(u'Power Density (mW cm^-2)')
    ax2.scatter(x_current[0:21],predict_power_235[0:21],color='green',label='Predicted Power Density ')
    ax2.plot(x_current[0:21],predict_power_real_235[0:21],color='blue')
    plt.legend()
    plt.savefig('%s CURVE 3.png' %(algorithm_name))
    print(voltage_result_list_235[0:21])
    print(data.iloc[235,27:48].values)
    print(predict_power_235[0:21])
    print(predict_power_real_235[0:21])
    
    fig4 = plt.figure()
    ax = fig4.add_subplot(111)
    ax.scatter(x_current[0:17],voltage_result_list_456[0:17],color='red',label='Predicted Voltage ')
    ax.plot(x_current[0:17],data.iloc[456,27:44].values,color='orange')
    ax.set_xlabel(u"Current(mA cm^-2)")
    ax.set_ylabel(u"Voltage(V)")
    ax2 = ax.twinx()
    ax2.set_ylabel(u'Power Density (mW cm^-2)')
    ax2.scatter(x_current[0:17],predict_power_456[0:17],color='green',label='Predicted Power Density ')
    ax2.plot(x_current[0:17],predict_power_real_456[0:17],color='blue')
    plt.legend()
    plt.savefig('%s CURVE 4.png' %(algorithm_name))
    print(voltage_result_list_456[0:17])
    print(data.iloc[456,27:44].values)
    print(predict_power_456[0:17])
    print(predict_power_real_456[0:17])
    
    fig5 = plt.figure()
    ax = fig5.add_subplot(111)
    ax.scatter(x_current[0:26],voltage_result_list_582[0:26],color='red',label='Predicted Voltage ')
    ax.plot(x_current[0:26],data.iloc[582,27:53].values,color='orange')
    ax.set_xlabel(u"Current(mA cm^-2)")
    ax.set_ylabel(u"Voltage(V)")
    ax2 = ax.twinx()
    ax2.set_ylabel(u'Power Density (mW cm^-2)')
    ax2.scatter(x_current[0:26],predict_power_582[0:26],color='green',label='Predicted Power Density ')
    ax2.plot(x_current[0:26],predict_power_real_582[0:26],color='blue')
    plt.legend()
    plt.savefig('%s CURVE 5.png' %(algorithm_name))
    print(voltage_result_list_582[0:26])
    print(data.iloc[582,27:53].values)
    print(predict_power_582[0:26])
    print(predict_power_real_582[0:26])

    fig6 = plt.figure()
    ax = fig6.add_subplot(111)
    ax.scatter(x_current[0:31],voltage_result_list_583[0:31],color='red',label='Predicted Voltage ')
    ax.plot(x_current[0:31],data.iloc[583,27:58].values,color='orange')
    ax.set_xlabel(u"Current(mA cm^-2)")
    ax.set_ylabel(u"Voltage(V)")
    ax2 = ax.twinx()
    ax2.set_ylabel(u'Power Density (mW cm^-2)')
    ax2.scatter(x_current[0:31],predict_power_583[0:31],color='green',label='Predicted Power Density ')
    ax2.plot(x_current[0:31],predict_power_real_583[0:31],color='blue')
    plt.legend()
    plt.savefig('%s CURVE 6.png' %(algorithm_name))
    print(voltage_result_list_583[0:31])
    print(data.iloc[583,27:58].values)
    print(predict_power_583[0:31])
    print(predict_power_real_583[0:31])
    
    print(corr_ann_235,corr_ann_456,corr_ann_381,corr_ann_619,corr_ann_582,corr_ann_583,rmse_val_235,rmse_val_456,rmse_val_381,rmse_val_619,rmse_val_582,rmse_val_583)
def try_different_method_xgboost(model,param,algorithm_name):
    voltage_result_list_each_235=[]
    voltage_result_list_each_381=[]
    voltage_result_list_each_456=[]
    voltage_result_list_each_619=[]
    voltage_result_list_each_582=[]
    voltage_result_list_each_583=[]
    for i in range(27,58,1):
        raw_param=standardized_data.iloc[:,0:27]
        raw_power=standardized_data.iloc[:,i]
        X_train=raw_param.drop(index=[235,381,456,619,582,583])
        y_train=raw_power.drop(index=[235,381,456,619,582,583])
        X_test_235=standardized_data.iloc[235,0:27]
        X_test_381=standardized_data.iloc[381,0:27]
        X_test_456=standardized_data.iloc[456,0:27]
        X_test_619=standardized_data.iloc[619,0:27]
        X_test_582=standardized_data.iloc[582,0:27]
        X_test_583=standardized_data.iloc[583,0:27]
        np.random.seed(1)
        grid = GridSearchCV(model,param_grid=param,cv=5)
        grid.fit(X_train,y_train)
        print('Best Regressor:',grid.best_params_,'Best Score:', grid.best_score_) 
        best_model=grid.best_estimator_
        best_model.fit(X_train.values,y_train.values)
        ###########get the prediction##########
        data_test_input_235=X_test_235
        data_test_input_235=pd.DataFrame(data_test_input_235)
        data_test_input_235=data_test_input_235.T
        data_test_param_235=data_test_input_235.values.astype(np.float32)
        predict_ann_235_each= best_model.predict(data_test_param_235)
        voltage_result_list_each_235.append(predict_ann_235_each[0])
        
        data_test_input_381=X_test_381
        data_test_input_381=pd.DataFrame(data_test_input_381)
        data_test_input_381=data_test_input_381.T
        data_test_param_381=data_test_input_381.values.astype(np.float32)
        predict_ann_381_each= best_model.predict(data_test_param_381)
        voltage_result_list_each_381.append(predict_ann_381_each[0])
        
        data_test_input_456=X_test_456
        data_test_input_456=pd.DataFrame(data_test_input_456)
        data_test_input_456=data_test_input_456.T
        data_test_param_456=data_test_input_456.values.astype(np.float32)
        predict_ann_456_each= best_model.predict(data_test_param_456)
        voltage_result_list_each_456.append(predict_ann_456_each[0])
        
        data_test_input_619=X_test_619
        data_test_input_619=pd.DataFrame(data_test_input_619)
        data_test_input_619=data_test_input_619.T
        data_test_param_619=data_test_input_619.values.astype(np.float32)
        predict_ann_619_each= best_model.predict(data_test_param_619)
        voltage_result_list_each_619.append(predict_ann_619_each[0])
        
        data_test_input_582=X_test_582
        data_test_input_582=pd.DataFrame(data_test_input_582)
        data_test_input_582=data_test_input_582.T
        data_test_param_582=data_test_input_582.values.astype(np.float32)
        predict_ann_582_each= best_model.predict(data_test_param_582)
        voltage_result_list_each_582.append(predict_ann_582_each[0])

        data_test_input_583=X_test_583
        data_test_input_583=pd.DataFrame(data_test_input_583)
        data_test_input_583=data_test_input_583.T
        data_test_param_583=data_test_input_583.values.astype(np.float32)
        predict_ann_583_each= best_model.predict(data_test_param_583)
        voltage_result_list_each_583.append(predict_ann_583_each[0])
        
    ###########transfer the prediction to voltage values##########235
    voltage_result_list_235=np.std(raw_power_global,axis=0).T.values*voltage_result_list_each_235+np.mean(raw_power_global,axis=0).T.values
    print(voltage_result_list_235)
    ###########computing output power##########
    x_current=np.arange(0,3100,100)
    predict_power_235=x_current*voltage_result_list_235
    predict_power_235=predict_power_235.T
    ###########real values in literature##########
    curve_real_235=data.iloc[235,27:]
    predict_power_real_235=x_current*data.iloc[235,27:].values
    ###########computing R2##########
    real_current_series_235=pd.Series(data.iloc[235,27:48].values)
    voltage_result_series_235=pd.Series(voltage_result_list_235[0:21])
    corr_ann_235 = round(voltage_result_series_235.corr(real_current_series_235), 4)
    rmse_val_235= rmse(data.iloc[235,27:48].values,voltage_result_list_235[0:21])
    print('ANN_235',corr_ann_235,'RMSE_235',rmse_val_235)   
    
    ###########transfer the prediction to voltage values##########381
    voltage_result_list_381=np.std(raw_power_global,axis=0).T.values*voltage_result_list_each_381+np.mean(raw_power_global,axis=0).T.values
    print(voltage_result_list_381)
    ###########computing output power##########
    x_current=np.arange(0,3100,100)
    predict_power_381=x_current*voltage_result_list_381
    predict_power_381=predict_power_381.T
    ###########real values in literature##########
    curve_real_381=data.iloc[381,27:]
    predict_power_real_381=x_current*data.iloc[381,27:].values
    ###########computing R2##########
    real_current_series_381=pd.Series(data.iloc[381,27:48].values)
    voltage_result_series_381=pd.Series(voltage_result_list_381[0:21])
    corr_ann_381 = round(voltage_result_series_381.corr(real_current_series_381), 4)
    rmse_val_381= rmse(data.iloc[381,27:48].values,voltage_result_list_381[0:21])
    print('ANN_381',corr_ann_381,'RMSE_381',rmse_val_381)    

    ###########transfer the prediction to voltage values##########456
    voltage_result_list_456=np.std(raw_power_global,axis=0).T.values*voltage_result_list_each_456+np.mean(raw_power_global,axis=0).T.values
    print(voltage_result_list_456)
    ###########computing output power##########
    x_current=np.arange(0,3100,100)
    predict_power_456=x_current*voltage_result_list_456
    predict_power_456=predict_power_456.T
    ###########real values in literature##########
    curve_real_456=data.iloc[456,27:]
    predict_power_real_456=x_current*data.iloc[456,27:].values
    ###########computing R2##########
    real_current_series_456=pd.Series(data.iloc[456,27:44].values)
    voltage_result_series_456=pd.Series(voltage_result_list_456[0:17])
    corr_ann_456 = round(voltage_result_series_456.corr(real_current_series_456), 4)
    rmse_val_456= rmse(data.iloc[456,27:44].values,voltage_result_list_456[0:17])  
    print('ANN_456',corr_ann_456,'RMSE_456',rmse_val_456)    

    ###########transfer the prediction to voltage values##########619
    voltage_result_list_619=np.std(raw_power_global,axis=0).T.values*voltage_result_list_each_619+np.mean(raw_power_global,axis=0).T.values
    print(voltage_result_list_619)
    ###########computing output power##########
    x_current=np.arange(0,3100,100)
    predict_power_619=x_current*voltage_result_list_619
    predict_power_619=predict_power_619.T
    ###########real values in literature##########
    curve_real_619=data.iloc[619,27:]
    predict_power_real_619=x_current*data.iloc[619,27:].values
    ###########computing R2##########
    real_current_series_619=pd.Series(data.iloc[619,27:48].values)
    voltage_result_series_619=pd.Series(voltage_result_list_619[0:21])
    corr_ann_619 = round(voltage_result_series_619.corr(real_current_series_619), 4)
    rmse_val_619= rmse(data.iloc[619,27:48].values,voltage_result_list_619[0:21])
    print('ANN_619',corr_ann_619,'RMSE_619',rmse_val_619)  
    
    voltage_result_list_582=np.std(raw_power_global,axis=0).T.values*voltage_result_list_each_582+np.mean(raw_power_global,axis=0).T.values
    print(voltage_result_list_582)
    ###########computing output power##########
    x_current=np.arange(0,3100,100)
    predict_power_582=x_current*voltage_result_list_582
    predict_power_582=predict_power_582.T
    ###########real values in literature##########
    curve_real_582=data.iloc[582,27:]
    predict_power_real_582=x_current*data.iloc[582,27:].values
    ###########computing R2##########
    real_current_series_582=pd.Series(data.iloc[582,27:53].values)
    voltage_result_series_582=pd.Series(voltage_result_list_582[0:26])
    corr_ann_582 = round(voltage_result_series_582.corr(real_current_series_582), 4)
    rmse_val_582= rmse(data.iloc[582,27:53].values,voltage_result_list_582[0:26])
    print('ANN_582',corr_ann_582,'RMSE_582',rmse_val_582)
    
    voltage_result_list_583=np.std(raw_power_global,axis=0).T.values*voltage_result_list_each_583+np.mean(raw_power_global,axis=0).T.values
    print(voltage_result_list_583)
    ###########computing output power##########
    x_current=np.arange(0,3100,100)
    predict_power_583=x_current*voltage_result_list_583
    predict_power_583=predict_power_583.T
    ###########real values in literature##########
    curve_real_583=data.iloc[583,27:]
    predict_power_real_583=x_current*data.iloc[583,27:].values
    ###########computing R2##########
    real_current_series_583=pd.Series(data.iloc[583,27:58].values)
    voltage_result_series_583=pd.Series(voltage_result_list_583[0:31])
    corr_ann_583 = round(voltage_result_series_583.corr(real_current_series_583), 4)
    rmse_val_583= rmse(data.iloc[583,27:58].values,voltage_result_list_583[0:31])
    print('ANN_583',corr_ann_583,'RMSE_583',rmse_val_583)


    ###########visualization##########
    fig1 = plt.figure()
    ax = fig1.add_subplot(111)
    ax.scatter(x_current[0:21],voltage_result_list_381[0:21],color='red',label='Predicted Voltage ')
    ax.plot(x_current[0:21],data.iloc[381,27:48].values,color='orange')
    ax.set_xlabel(u"Current(mA cm^-2)")
    ax.set_ylabel(u"Voltage(V)")
    ax2 = ax.twinx()
    ax2.set_ylabel(u'Power Density (mW cm^-2)')
    ax2.scatter(x_current[0:21],predict_power_381[0:21],color='green',label='Predicted Power Density ')
    ax2.plot(x_current[0:21],predict_power_real_381[0:21],color='blue')
    plt.legend()
    plt.savefig('%s CURVE 1.png' %(algorithm_name))
    print(voltage_result_list_381[0:21])
    print(data.iloc[381,27:48].values)
    print(predict_power_381[0:21])
    print(predict_power_real_381[0:21])
    
    fig2 = plt.figure()
    ax = fig2.add_subplot(111)
    ax.scatter(x_current[0:21],voltage_result_list_619[0:21],color='red',label='Predicted Voltage ')
    ax.plot(x_current[0:21],data.iloc[619,27:48].values,color='orange')
    ax.set_xlabel(u"Current(mA cm^-2)")
    ax.set_ylabel(u"Voltage(V)")
    ax2 = ax.twinx()
    ax2.set_ylabel(u'Power Density (mW cm^-2)')
    ax2.scatter(x_current[0:21],predict_power_619[0:21],color='green',label='Predicted Power Density ')
    ax2.plot(x_current[0:21],predict_power_real_619[0:21],color='blue')
    plt.legend()
    plt.savefig('%s CURVE 2.png' %(algorithm_name))
    print(voltage_result_list_619[0:21])
    print(data.iloc[619,27:48].values)
    print(predict_power_619[0:21])
    print(predict_power_real_619[0:21])

    fig3 = plt.figure()
    ax = fig3.add_subplot(111)
    ax.scatter(x_current[0:21],voltage_result_list_235[0:21],color='red',label='Predicted Voltage ')
    ax.plot(x_current[0:21],data.iloc[235,27:48].values,color='orange')
    ax.set_xlabel(u"Current(mA cm^-2)")
    ax.set_ylabel(u"Voltage(V)")
    ax2 = ax.twinx()
    ax2.set_ylabel(u'Power Density (mW cm^-2)')
    ax2.scatter(x_current[0:21],predict_power_235[0:21],color='green',label='Predicted Power Density ')
    ax2.plot(x_current[0:21],predict_power_real_235[0:21],color='blue')
    plt.legend()
    plt.savefig('%s CURVE 3.png' %(algorithm_name))
    print(voltage_result_list_235[0:21])
    print(data.iloc[235,27:48].values)
    print(predict_power_235[0:21])
    print(predict_power_real_235[0:21])
    
    fig4 = plt.figure()
    ax = fig4.add_subplot(111)
    ax.scatter(x_current[0:17],voltage_result_list_456[0:17],color='red',label='Predicted Voltage ')
    ax.plot(x_current[0:17],data.iloc[456,27:44].values,color='orange')
    ax.set_xlabel(u"Current(mA cm^-2)")
    ax.set_ylabel(u"Voltage(V)")
    ax2 = ax.twinx()
    ax2.set_ylabel(u'Power Density (mW cm^-2)')
    ax2.scatter(x_current[0:17],predict_power_456[0:17],color='green',label='Predicted Power Density ')
    ax2.plot(x_current[0:17],predict_power_real_456[0:17],color='blue')
    plt.legend()
    plt.savefig('%s CURVE 4.png' %(algorithm_name))
    print(voltage_result_list_456[0:17])
    print(data.iloc[456,27:44].values)
    print(predict_power_456[0:17])
    print(predict_power_real_456[0:17])
    
    fig5 = plt.figure()
    ax = fig5.add_subplot(111)
    ax.scatter(x_current[0:26],voltage_result_list_582[0:26],color='red',label='Predicted Voltage ')
    ax.plot(x_current[0:26],data.iloc[582,27:53].values,color='orange')
    ax.set_xlabel(u"Current(mA cm^-2)")
    ax.set_ylabel(u"Voltage(V)")
    ax2 = ax.twinx()
    ax2.set_ylabel(u'Power Density (mW cm^-2)')
    ax2.scatter(x_current[0:26],predict_power_582[0:26],color='green',label='Predicted Power Density ')
    ax2.plot(x_current[0:26],predict_power_real_582[0:26],color='blue')
    plt.legend()
    plt.savefig('%s CURVE 5.png' %(algorithm_name))
    print(voltage_result_list_582[0:26])
    print(data.iloc[582,27:53].values)
    print(predict_power_582[0:26])
    print(predict_power_real_582[0:26])

    fig6 = plt.figure()
    ax = fig6.add_subplot(111)
    ax.scatter(x_current[0:31],voltage_result_list_583[0:31],color='red',label='Predicted Voltage ')
    ax.plot(x_current[0:31],data.iloc[583,27:58].values,color='orange')
    ax.set_xlabel(u"Current(mA cm^-2)")
    ax.set_ylabel(u"Voltage(V)")
    ax2 = ax.twinx()
    ax2.set_ylabel(u'Power Density (mW cm^-2)')
    ax2.scatter(x_current[0:31],predict_power_583[0:31],color='green',label='Predicted Power Density ')
    ax2.plot(x_current[0:31],predict_power_real_583[0:31],color='blue')
    plt.legend()
    plt.savefig('%s CURVE 6.png' %(algorithm_name))
    print(voltage_result_list_583[0:31])
    print(data.iloc[583,27:58].values)
    print(predict_power_583[0:31])
    print(predict_power_real_583[0:31])
    
    print(corr_ann_235,corr_ann_456,corr_ann_381,corr_ann_619,corr_ann_582,corr_ann_583,rmse_val_235,rmse_val_456,rmse_val_381,rmse_val_619,rmse_val_582,rmse_val_583)
####import machine learning algorithms packages and define the corresponding models####
####Decision Tree####
from sklearn import tree
from sklearn.tree import DecisionTreeRegressor
from sklearn import metrics
model_DecisionTreeRegressor = tree.DecisionTreeRegressor()
####Linear Regression####
from sklearn import linear_model
model_LinearRegression = linear_model.LinearRegression()
####Support Vector Regressor####
from sklearn import svm
model_SVR = svm.SVR()
####K Nearest Neighbor####
from sklearn import neighbors
model_KNeighborsRegressor = neighbors.KNeighborsRegressor()
####Random Forest####
from sklearn import ensemble
model_RandomForestRegressor = ensemble.RandomForestRegressor()
####Adaboost####
from sklearn import ensemble
model_AdaBoostRegressor = ensemble.AdaBoostRegressor()
####GBRT####
from sklearn import ensemble
model_GradientBoostingRegressor = ensemble.GradientBoostingRegressor()
####Bagging Regressor ####
from sklearn.ensemble import BaggingRegressor
model_BaggingRegressor = BaggingRegressor()
####Extra Tree Regressor####
from sklearn.tree import ExtraTreeRegressor
model_ExtraTreeRegressor = ExtraTreeRegressor()
####Bayesian Ridge####
from sklearn.linear_model import BayesianRidge
model_BayesianRidge =BayesianRidge()
####Gaussian Process####
from sklearn.gaussian_process import GaussianProcessRegressor
model_GaussianProcessRegressor=GaussianProcessRegressor()
####SGD####
from sklearn.linear_model.stochastic_gradient import SGDRegressor
model_SGDRegressor=SGDRegressor()
####Ridge Regressor####
from sklearn.linear_model import Ridge,RidgeCV
model_RidgeRegressor=Ridge()
####Kernel Ridge####
from sklearn.kernel_ridge import KernelRidge
model_KernelRidgeRegressor=KernelRidge()
####XGBoost####
import xgboost as xgb
model_XGboostRegressor=xgb.XGBRegressor()
####CatBoost####
import catboost
model_CatboostRegressor=catboost.CatBoostRegressor()
####LightGBM####
import lightgbm
model_LGBMRegressor=lightgbm.LGBMRegressor()
####Lasso Regressor####
from sklearn import linear_model
model_LassoRegressor=linear_model.LassoCV()
####Elastic Net####
from sklearn import linear_model
model_ElasticNetRegressor=linear_model.ElasticNetCV()
####LARS####
from sklearn import linear_model
model_LARSRegressor=linear_model.LarsCV()
####LassoLARS####
from sklearn import linear_model
model_LARSLassoRegressor=linear_model.LassoLarsCV()
####OMP####
from sklearn import linear_model
model_OMPRegressor=linear_model.OrthogonalMatchingPursuitCV()
####ARD####
from sklearn import linear_model
model_ARDRegressor=linear_model.ARDRegression()
####PAR####
from sklearn import linear_model
model_PARRegressor=linear_model.PassiveAggressiveRegressor()
####RANSAC####
from sklearn import linear_model
model_RANSACRegressor=linear_model.RANSACRegressor()
####TSR####
from sklearn import linear_model
model_TSRRegressor=linear_model.TheilSenRegressor()
####Huber####
from sklearn import linear_model
model_HuberRegressor=linear_model.HuberRegressor()
####Polynomial####
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
model_PolynomialRegressor=Pipeline([('poly', PolynomialFeatures(degree=4)),('linear', LinearRegression(fit_intercept=False))])
####Linear Support Vector Regressor####
from sklearn import svm
model_LinearSVR = svm.LinearSVR()
####Nu Support Vector Regressor####
from sklearn import svm
model_NuSVR = svm.NuSVR()
####Voting Regressor ####
from sklearn.ensemble import VotingRegressor

# Build all top scoring models to test them for GridSearch Voting Regressor
rf_estimator = ensemble.RandomForestRegressor(max_depth=15, n_estimators=400,max_features='auto',min_samples_leaf=2, min_samples_split=2)
ada_estimator=ensemble.AdaBoostRegressor(learning_rate=1,loss='square',n_estimators=1000)
knn_estimator=neighbors.KNeighborsRegressor(algorithm='ball_tree',leaf_size=2,n_neighbors=8,weights='distance')
kernel_ridge_estimator=KernelRidge(alpha=1,coef0=0.01,degree=2,gamma=0.1,kernel='rbf')
gb_estimator=ensemble.GradientBoostingRegressor(criterion='friedman_mse',learning_rate=0.1,loss='huber',max_depth=10,max_features='auto',min_samples_split=8,n_estimators=200,subsample=0.6)
catboost_estimator=catboost.CatBoostRegressor(learning_rate=0.1,max_depth=5,n_estimators=800,subsample=0.5,verbose=0)
lightboost_estimator=lightgbm.LGBMRegressor(reg_alpha=0.1,reg_lambda=0.1,learning_rate=0.01,max_depth=5,n_estimators=4000,subsample=0.5)

# Make list of all models
estimators_list = [
              ('rf', rf_estimator),
                  ('ada',ada_estimator),
                  ('knn',knn_estimator),
                  ('kernel_ridge',kernel_ridge_estimator),
                  ('gb',gb_estimator),
                  ('cat',catboost_estimator),
                  ('light',lightboost_estimator)]
model_VotingRegressor=VotingRegressor(estimators_list)
####Stacking Regressor ####
from sklearn.ensemble import StackingRegressor
model_StackingRegressor = StackingRegressor(estimators_list)

D:\anaconda\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
D:\anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
D:\anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
D:\anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)typ

each cell below could be conducted to get the corresponding result of the machine learning algoritm you want independently

In [ ]:
seed=2
np.random.seed(seed)
####Decision Tree####
###########defining the parameters dictionary##########
param = {'criterion':['mse','mae','friedman_mse'],
         'max_depth':range(5,15),
         'splitter':['random','best'],
         'min_samples_split':range(2,10),
         'max_features':["auto", "sqrt", "log2"]
       }
try_different_method(model_DecisionTreeRegressor,param,'Decision Tree Regressor')

In [ ]:
seed=2
np.random.seed(seed)
####Support Vector Regressor####
###########defining the parameters dictionary##########
param = {'kernel':['linear', 'poly', 'rbf'],'max_iter':[100,200,300,400,500,600,700,800,1000,1100,1200,1300,1400,1500],
         'degree':[2,3,4],
         'gamma':['scale','auto'],
         'epsilon':[0.001,0.01,0.1,0.3,0.5,0.7,1],'coef0':[100,200,300,400,500,600,700,800,1000,1100,1200,1300,1400,1500]
       }
try_different_method(model_SVR,param,'Support Vector Regressor')

In [ ]:
seed=2
np.random.seed(seed)
####K Nearest Neighbor####
###########defining the parameters dictionary##########
param = {'n_neighbors':range(1,10),'weights':['uniform','distance'],'algorithm':['auto', 'ball_tree', 'kd_tree', 'brute'],
         'leaf_size':[2,10,20,30,40,50,100],
         'p':range(1,10)
       }

try_different_method(model_KNeighborsRegressor,param,'KNeighbors Regressor')

In [ ]:
seed=2
np.random.seed(seed)
####Random Forest####
###########defining the parameters dictionary##########
param = {'n_estimators':[200,400,800],
         'max_depth':[5,10,15],
         'min_samples_split':[2,4,6,8],
         'min_samples_leaf':[2,4,8],
         'max_features':['auto','sqrt','log2']

       }

try_different_method(model_RandomForestRegressor,param,'Random Forest Regressor')

In [ ]:
seed=2
np.random.seed(seed)
####Gradient Boost####
###########defining the parameters dictionary##########
param = {'n_estimators':[200,400,800],
         'max_depth':[5,10,15],
         'min_samples_split':[2,4,6,8],
         'max_features':['auto','sqrt','log2'],
         'loss':['ls', 'lad', 'huber', 'quantile'],
         'learning_rate':[0.01,0.1,0.2],
         'criterion':['friedman_mse'],
         'subsample':[0.6,0.8,1]
       }
algorithm_name='Gradient Boost Regressor'
try_different_method(model_GradientBoostingRegressor,param,'Gradient Boost Regressor')

In [ ]:
seed=2
np.random.seed(seed)
####SGD####
###########defining the parameters dictionary##########
param = {'loss':['squared_loss', 'huber','epsilon_insensitive','squared_epsilon_insensitive'],
         'penalty':['l2', 'elasticnet'],
         'fit_intercept':[True,False],
         'max_iter':[500,1000,2000],
         'epsilon':[0.001,0.01,0.1,1],
         'learning_rate':['invscaling','constant', 'optimal', 'adaptive']
       }
try_different_method(model_SGDRegressor,param,'Stochastic Gradient Descent Regressor')

In [ ]:
seed=2
np.random.seed(seed)
####Bagging Regressor####
###########defining the parameters dictionary##########
param = {'n_estimators':[10,50,100,200,400],
         'max_samples':[5,10,50,100],
         'max_features':[5,10,15,20,27]
       }

try_different_method(model_BaggingRegressor,param,'Bagging Regressor')

In [ ]:
seed=2
np.random.seed(seed)
####extreme tree####
###########defining the parameters dictionary##########
param = {
         'criterion' : ["mse", "friedman_mse", "mae"],
        'splitter' : [ "best",'random'],
       'max_depth':range(5,15),
       'min_samples_split':[2,4,6,8],
         'min_samples_leaf':[1,2,4,8]
       }

try_different_method(model_ExtraTreeRegressor,param,'Extra Tree Regressor')

In [ ]:
seed=2
np.random.seed(seed)
####Kernel Ridge####
###########defining the parameters dictionary##########
param = {
         'alpha':[1,1.5,2],
        'kernel':['rbf', 'linear'],
        'gamma':[0.1,0.5,1,2],
        'degree':[2,3,4],
        'coef0':[0.01,0.1,0.5,1,10,50,100,200,500,1000]
         
       }

try_different_method(model_KernelRidgeRegressor,param,'Kernel Ridge Regressor')

In [2]:
seed=2
np.random.seed(seed)
####Ridge Regressor####
###########defining the parameters dictionary##########
param = {
         'alpha':[0.1,0.2,0.4,0.8,1,1.6,2,4,8],
        'solver':['saga','auto']  
       }

try_different_method(model_RidgeRegressor,param,'Ridge Regressor')

Best Regressor: {'alpha': 8, 'solver': 'saga'} Best Score: -0.23622929585837177
(1, 27)
Best Regressor: {'alpha': 8, 'solver': 'saga'} Best Score: -0.09853275653710811
(1, 27)
Best Regressor: {'alpha': 8, 'solver': 'saga'} Best Score: -0.15046777198933736
(1, 27)
Best Regressor: {'alpha': 8, 'solver': 'saga'} Best Score: -0.1955273882497926
(1, 27)


KeyboardInterrupt: 

In [ ]:
seed=2
np.random.seed(seed)
####Bayesian Rdige Regressor####
###########defining the parameters dictionary##########
param = {
         'n_iter':[50,100,200,400,800],  
       }

try_different_method(model_BayesianRidge,param,'Bayesian Ridge Regressor')

In [ ]:
seed=2
np.random.seed(seed)
####XGboost####
###########defining the parameters dictionary##########
param = {
'learning_rate':[0.01,0.02,0.05,0.1,0.2,1],
   'n_estimators':[100,200,400,1000,2000,4000,8000],
    'max_depth':[5,7,9,11],
    'subsample':[0.5,0.6,0.7,0.8,0.9],
    'reg_lambda':[0.1,0.01],#reg_lambda in cloud version
    'reg_alpha':[0.1,0.01]#reg_alpha in cloud version
       }

try_different_method_xgboost(model_XGboostRegressor,param,'XGBoost Regressor')

In [ ]:
seed=2
np.random.seed(seed)
####Adaboost####
###########defining the parameters dictionary##########
param = {
    'n_estimators':[300,500,700,1000],
         'learning_rate':[0.01,0.1,1,2],
         'loss':[ 'square']
       }

try_different_method(model_AdaBoostRegressor,param,'AdaBoost Regressor')

In [ ]:
seed=2
np.random.seed(seed)
####Linear Regressor####
param={}
algorithm_name='Linear Regressor'
try_different_method(model_LinearRegression,param,'Linear Regression')

In [ ]:
seed=2
np.random.seed(seed)
####Gaussian Process Regressor####
param={}
try_different_method(model_GaussianProcessRegressor,param,'Gaussian Process Regressor')

In [ ]:
seed=2
np.random.seed(seed)
####catboost####
###########defining the parameters dictionary##########

param = {
         'learning_rate':[0.01,0.02,0.1,0.2],
       'n_estimators':[100,200,400,800],
    'max_depth':[3,5,7,9,11],
    'subsample':[0.5,0.7,0.9],
    'verbose':[0]
       }

try_different_method(model_CatboostRegressor,param,'Cat Boost Regressor')

In [ ]:
seed=2
np.random.seed(seed)
####lightboost####
###########defining the parameters dictionary##########
param = {
         'learning_rate':[0.01,0.1,1],
   'n_estimators':[100,200,400,1000,2000,4000,8000],
    'max_depth':[5,7,9,11],
    'subsample':[0.5,0.6,0.7,0.8,0.9],
    'lambda':[0.1,0.01],
    'alpha':[0.1,0.01]
       }

try_different_method(model_LGBMRegressor,param,'Light Boost Regressor')

In [ ]:
seed=2
np.random.seed(seed)
####Lasso Regressor####
from sklearn import linear_model
param = {'eps':[0.001,0.0001,0.01,0.1],
         'max_iter':[50,100,200,400,800,1600],  
         'n_alphas':[100,200,400,800,1600]
         
       }

try_different_method(model_LassoRegressor,param,'Lasso Regressor')

In [ ]:
seed=2
np.random.seed(seed)
####ElasticNet Regressor####
from sklearn import linear_model
param = {'eps':[0.001,0.0001,0.01,0.1],
         'max_iter':[50,100,200,400,800,1600],  
         'n_alphas':[100,200,400,800,1600] 
       }

try_different_method(model_ElasticNetRegressor,param,'Elastic Net Regressor')

In [ ]:
seed=2
np.random.seed(seed)
####LARS Regressor####
from sklearn import linear_model
param = {
         'max_iter':[500,1000,2000,4000,8000,16000],  
         'max_n_alphas':[500,1000,2000,] 
       }

try_different_method(model_LARSRegressor,param,'LARS Regressor')

In [ ]:
seed=2
np.random.seed(seed)
####OMP Regressor####
from sklearn import linear_model
param = {
         'max_iter':[5,10,20,30,50]
       }

try_different_method(model_OMPRegressor,param,'OMP Regressor')

In [ ]:
seed=2
np.random.seed(seed)
####ARD Regressor####
from sklearn import linear_model
param = {
         'n_iter':[150,300,600,1200,2400],
    'alpha_1':[1e-7,1e-6,1e-5],
    'alpha_2':[1e-7,1e-6,1e-5],
    'lambda_1':[1e-7,1e-6,1e-5],
    'lambda_2':[1e-7,1e-6,1e-5]
       }

try_different_method(model_ARDRegressor,param,'ARD Regressor')

In [ ]:
seed=2
np.random.seed(seed)
####PAR Regressor####
from sklearn import linear_model
param = {
         'C':[0.1,1,10],
    'max_iter':[500,1000,1500,2000],
    'tol':[0.0001,0.001,0.01,0.1],
    'validation_fraction':[0.1,0.3,0.5],
    'epsilon':[0.001,0.01,0.1]
       }

try_different_method(model_PARRegressor,param,'PAR Regressor')

In [ ]:
seed=2
np.random.seed(seed)
####TSR Regressor####
from sklearn import linear_model
param = {
         'max_subpopulation':[1000,5000,10000,20000,50000,100000],
    'max_iter':[150,300,500,1000,3000,5000],
  
       }

try_different_method(model_TSRRegressor,param,'TSRegressor')

In [ ]:
seed=2
np.random.seed(seed)
####Huber Regressor####
from sklearn import linear_model
param = {
         'alpha':[0.00001,0.0001,0.001,0.01,0.1],
    'max_iter':[100,400,800,1600,3200],
  
       }

try_different_method(model_HuberRegressor,param,'Huber Regressor')

In [ ]:
seed=2
np.random.seed(seed)
####Polynomial Regressor####
from sklearn import linear_model
param={}

try_different_method(model_PolynomialRegressor,param,'Polynomial Regressor')

In [ ]:
seed=2
np.random.seed(seed)
####Linaer Support Vector Regressor####
###########defining the parameters dictionary##########
param = {'max_iter':[100,200,300,400,500,600,700,800,1000,1100,1200,1300,1400,1500,1600,1700,1800,1900,2000],
         'tol':[0.0001,0.001,0.01,0.1],
         'epsilon':[0,0.001,0.01,0.1,0.3,0.5,0.7,1]
       }

try_different_method(model_LinearSVR,param,'Linear Support Vector Regressor')

In [ ]:
seed=2
np.random.seed(seed)
####Nu Support Vector Regressor####
###########defining the parameters dictionary##########
param = {'kernel':['linear', 'poly', 'rbf'],
         'degree':[2,3,4],
         'gamma':['scale','auto'],
         'coef0':[0,10,100,200,400,800,1600]
       }

try_different_method(model_NuSVR,param,'NuSupport Vector Regressor')

In [ ]:
seed=2
np.random.seed(seed)
####Voting Regressor####
param={}
algorithm_name='Voting Regressor'
try_different_method(model_VotingRegressor,param,'Voting Regressor')

In [ ]:
seed=2
np.random.seed(seed)
####Stacking Regressor####
param={}
algorithm_name='Stacking Regressor'
try_different_method(model_StackingRegressor,param,'Stacking Regressor')

In [ ]:
seed=2
np.random.seed(seed)
####LARS Lasso Regressor####
from sklearn import linear_model
param = {
         'max_iter':[500,1000,2000,4000,8000,16000],  
         'max_n_alphas':[500,1000,2000,] 
       }

try_different_method(model_LARSLassoRegressor,param,'LARS Lasso Regressor')